# Training our model on our feature dataset in prep for insertion into A* heuristic

### Starting with a support vector machine

In [44]:
import pandas as pd
data = pd.read_csv('attributes.csv')
data.head()

,cost,heaviest queen weight,location of heaviest queen,lightest queen weight,location of lightest queen,initial conflicts,n,average value including 0,lightest in conflict,heaviest in conflict,average in conflict,smallest d in conflict,largest d in conflict,average d in conflict,row with most queens,row with least queens,num empty rows,most queens in row,least queens in row,avg queens in nonzero row
0,37,8,"(3, 1)",1,"(0, 2)",6,4,1.0000,1,8,4.166667,1.414214,2.828427,2.357023,0,0,0,1,1,1
1,8,7,"(0, 2)",2,"(2, 3)",0,4,1.2500,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,1,1,1
2,82,6,"(1, 0)",3,"(3, 1)",2,4,1.1250,3,5,4.000000,1.414214,1.414214,1.414214,0,0,0,1,1,1
3,119,7,"(2, 1)",3,"(1, 2)",3,4,1.3125,3,7,5.000000,1.414214,4.242641,2.357023,0,0,0,1,1,1
4,72,8,"(0, 1)",6,"(3, 2)",0,4,1.8125,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,1,1,1


In [45]:
Y = data['cost']
Y.head()

0     37
1      8
2     82
3    119
4     72
Name: cost, dtype: int64

In [46]:
X = data.loc[:, data.columns != 'cost']
X.head()

,heaviest queen weight,location of heaviest queen,lightest queen weight,location of lightest queen,initial conflicts,n,average value including 0,lightest in conflict,heaviest in conflict,average in conflict,smallest d in conflict,largest d in conflict,average d in conflict,row with most queens,row with least queens,num empty rows,most queens in row,least queens in row,avg queens in nonzero row
0,8,"(3, 1)",1,"(0, 2)",6,4,1.0000,1,8,4.166667,1.414214,2.828427,2.357023,0,0,0,1,1,1
1,7,"(0, 2)",2,"(2, 3)",0,4,1.2500,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,1,1,1
2,6,"(1, 0)",3,"(3, 1)",2,4,1.1250,3,5,4.000000,1.414214,1.414214,1.414214,0,0,0,1,1,1
3,7,"(2, 1)",3,"(1, 2)",3,4,1.3125,3,7,5.000000,1.414214,4.242641,2.357023,0,0,0,1,1,1
4,8,"(0, 1)",6,"(3, 2)",0,4,1.8125,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,1,1,1


### Our location features won't be accepted into an svc so we need to convert those to floats in the format (3,1) -> 3.1 and (0,2) -> 0.2 for simplicity.

In [47]:
def convert_float(inp):
    inp = inp.replace("(", "")
    inp = inp.replace(")", "")
    splitted_data = inp.split(",")
    temp = str(splitted_data[0] + '.' + splitted_data[1])
    temp = temp.replace(" ", "")
    return temp
X['location of heaviest queen'] = X['location of heaviest queen'].apply(lambda x: convert_float(x))
X['location of lightest queen'] = X['location of lightest queen'].apply(lambda x: convert_float(x))

C:\Users\Spencer\AppData\Local\Temp\ipykernel_11616\2872941786.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['location of heaviest queen'] = X['location of heaviest queen'].apply(lambda x: convert_float(x))
C:\Users\Spencer\AppData\Local\Temp\ipykernel_11616\2872941786.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['location of lightest queen'] = X['location of lightest queen'].apply(lambda x: convert_float(x))


In [48]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
clf = SVC(kernel='linear')
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.12288613, 0.11048478, 0.11386697, 0.11625282, 0.11625282,
       0.11173815, 0.11851016, 0.11286682, 0.10948081, 0.1027088 ])

### Okay that went poorly

In [51]:
X.describe()

,heaviest queen weight,lightest queen weight,initial conflicts,n,average value including 0,lightest in conflict,heaviest in conflict,average in conflict,smallest d in conflict,largest d in conflict,average d in conflict,row with most queens,row with least queens,num empty rows,most queens in row,least queens in row,avg queens in nonzero row
count,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.000000,8863.0,8863.0,8863.0,8863.0,8863.0,8863.0
mean,6.989168,2.039603,3.989281,4.545075,1.011267,2.491933,5.732822,4.105381,1.323900,2.606474,1.901665,0.0,0.0,0.0,1.0,1.0,1.0
std,1.204959,1.214665,2.817394,0.650828,0.280707,1.848749,2.436541,1.893297,0.487685,1.486165,0.868307,0.0,0.0,0.0,0.0,0.0,0.0
min,2.000000,1.000000,0.000000,4.000000,0.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,1.0
25%,6.000000,1.000000,2.000000,4.000000,0.812500,1.000000,5.000000,3.000000,1.414214,1.414214,1.414214,0.0,0.0,0.0,1.0,1.0,1.0
50%,7.000000,2.000000,3.000000,4.000000,1.000000,2.000000,7.000000,4.333333,1.414214,2.828427,1.885618,0.0,0.0,0.0,1.0,1.0,1.0
75%,8.000000,3.000000,6.000000,5.000000,1.187500,4.000000,8.000000,5.333333,1.414214,4.242641,2.357023,0.0,0.0,0.0,1.0,1.0,1.0
max,8.000000,8.000000,20.000000,8.000000,2.000000,8.000000,8.000000,8.000000,8.485281,8.485281,8.485281,0.0,0.0,0.0,1.0,1.0,1.0


### We'll try again but drop the rows 'row with most queens' and onwards since those are just constants in this dataset

In [54]:
X = X.drop(columns=['row with most queens', 'row with least queens', 'num empty rows', 'most queens in row', 'least queens in row', 'avg queens in nonzero row'])
X.head()

,heaviest queen weight,location of heaviest queen,lightest queen weight,location of lightest queen,initial conflicts,n,average value including 0,lightest in conflict,heaviest in conflict,average in conflict,smallest d in conflict,largest d in conflict,average d in conflict
0,8,3.1,1,0.2,6,4,1.0000,1,8,4.166667,1.414214,2.828427,2.357023
1,7,0.2,2,2.3,0,4,1.2500,0,0,0.000000,0.000000,0.000000,0.000000
2,6,1.0,3,3.1,2,4,1.1250,3,5,4.000000,1.414214,1.414214,1.414214
3,7,2.1,3,1.2,3,4,1.3125,3,7,5.000000,1.414214,4.242641,2.357023
4,8,0.1,6,3.2,0,4,1.8125,0,0,0.000000,0.000000,0.000000,0.000000


### And try again...

In [55]:
clf = SVC(kernel='linear')
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.12288613, 0.11048478, 0.11386697, 0.11625282, 0.11625282,
       0.11173815, 0.11851016, 0.11286682, 0.10948081, 0.10383747])

### Neglibible change. Might as well have kept those features

### What about a non-linear relationship?

In [56]:
clf = SVC(kernel='poly')
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.12965051, 0.11612176, 0.11273957, 0.11738149, 0.12641084,
       0.11963883, 0.11851016, 0.11399549, 0.12528217, 0.11851016])

### Also not great

### Let's try a random forest with depth of 10 for starters

In [57]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10)
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.19729425, 0.19278467, 0.18714769, 0.19638826, 0.19074492,
       0.1489842 , 0.1489842 , 0.13544018, 0.15349887, 0.13205418])

In [ ]:
avg = sum(scores) / len(scores)
avg

### Immediately better. How about depth of 20 to allow more branches

In [66]:
clf = RandomForestClassifier(max_depth=20)
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.22886133, 0.21307779, 0.22886133, 0.23250564, 0.21218962,
       0.17042889, 0.16478555, 0.13769752, 0.15349887, 0.12979684])

In [67]:
model = RandomForestClassifier(max_depth=20)
clf.fit(X, Y)

RandomForestClassifier(max_depth=20)

### Let's save this one after running all the others

In [68]:
import pickle
#save model
with open('model.pkl','wb') as f:
  pickle.dump(model,f)

In [59]:
avg = sum(scores) / len(scores)
avg

0.18694435042410948

### More depth?

In [60]:
clf = RandomForestClassifier(max_depth=30)
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.22096956, 0.21758737, 0.22773393, 0.24379233, 0.21218962,
       0.17042889, 0.15688488, 0.13092551, 0.15124153, 0.12866817])

In [61]:
avg = sum(scores) / len(scores)
avg

0.1860421793602602

### 20 depth vs 30 depth didn't change much, it actually did ~.09% worse

### Gaussian Naive Bayes?

In [64]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


array([0.09357384, 0.09131905, 0.08229989, 0.09255079, 0.08916479,
       0.08013544, 0.08577878, 0.07674944, 0.08465011, 0.08126411])

### Garbage...neural nets?

In [65]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
scores = cross_val_score(clf, X, Y, cv=10)
scores

C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Spencer\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs",

array([0.06087937, 0.04847802, 0.04735062, 0.04740406, 0.03837472,
       0.05417607, 0.05417607, 0.03837472, 0.05417607, 0.03950339])

### 20 depth random forest seems to be the best